<a href="https://colab.research.google.com/github/omkar123katare/Capstone_2_Retail_sales_prediction/blob/main/retail_sales__predicton_capstone_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Sales Prediction : Predicting sales of a major store chain Rossmann</u></b>

## <b> Problem Description </b>

### Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

### You are provided with historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.

## <b> Data Description </b>

### <b>Rossmann Stores Data.csv </b> - historical data including Sales
### <b>store.csv </b> - supplemental information about the stores


### <b><u>Data fields</u></b>
### Most of the fields are self-explanatory. The following are descriptions for those that aren't.

* #### Id - an Id that represents a (Store, Date) duple within the test set
* #### Store - a unique Id for each store
* #### Sales - the turnover for any given day (this is what you are predicting)
* #### Customers - the number of customers on a given day
* #### Open - an indicator for whether the store was open: 0 = closed, 1 = open
* #### StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
* #### SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
* #### StoreType - differentiates between 4 different store models: a, b, c, d
* #### Assortment - describes an assortment level: a = basic, b = extra, c = extended
* #### CompetitionDistance - distance in meters to the nearest competitor store
* #### CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
* #### Promo - indicates whether a store is running a promo on that day
* #### Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
* #### Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
* #### PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

# Step 1 : Business Problem and KPI 

# Step 2: Translating Business problem into data scinece Problem

##**Importing Libraries**

In [146]:
import pandas as pd

## **Mounting the drive and Importing Data from there**

In [147]:
# importing drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [148]:
# import rossman store data from drive 
rossmann_store_data=pd.read_csv('/content/drive/MyDrive/Data Science Projects-AlmaBetter/Capstone_2_store_sales_prediction/Rossmann Stores Data.csv')
# import store data from drive 
store_df= pd.read_csv("/content/drive/MyDrive/Data Science Projects-AlmaBetter/Capstone_2_store_sales_prediction/store.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [149]:
# to check shape of both data which gives number of rows and columns in each data 
rossmann_store_data.shape, store_df.shape

((1017209, 9), (1115, 10))

##**Data Exploration**

# Steps involved in Data Exploration and Preparation
1. Variable Identification
2. Univariate Analysis
3. Bi-variate Analysis
4. Missing values treatment
5. Outlier treatment
6. Variable transformation
7. Variable creation

In [150]:
# Merging both the dataset on store columnn because it present on both the dataset
df_raw = pd.merge(rossmann_store_data,store_df , on = 'Store', how='left' )

In [151]:
df_raw.shape

(1017209, 18)

In [152]:
df_raw_copy=df_raw.copy()

In [153]:
% pip install inflection

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Renaming the column names

In [154]:
import inflection

In [155]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo','StateHoliday', 'SchoolHoliday', 'StoreType',
            'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 
            'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']

# creating list of new names with a snakecase pattern

cols_new = []
for elem in cols_old:
  cols_new.append(inflection.underscore(elem))
print(cols_new)

['store', 'day_of_week', 'date', 'sales', 'customers', 'open', 'promo', 'state_holiday', 'school_holiday', 'store_type', 'assortment', 'competition_distance', 'competition_open_since_month', 'competition_open_since_year', 'promo2', 'promo2_since_week', 'promo2_since_year', 'promo_interval']


In [156]:
df_raw.columns=cols_new
df_raw_copy.columns=cols_new

In [157]:
print(f'New column names in the merged dataframe are {list(df_raw.columns)}')

New column names in the merged dataframe are ['store', 'day_of_week', 'date', 'sales', 'customers', 'open', 'promo', 'state_holiday', 'school_holiday', 'store_type', 'assortment', 'competition_distance', 'competition_open_since_month', 'competition_open_since_year', 'promo2', 'promo2_since_week', 'promo2_since_year', 'promo_interval']


### Data Dimensions

In [158]:
print(f'Number of rows in the merged dataset are {df_raw.shape[0]}')
print(f'Number of columns in the merged dataset are {df_raw.shape[1]}')

Number of rows in the merged dataset are 1017209
Number of columns in the merged dataset are 18


### Column Data types

In [159]:
df_raw.head(5)

,store,day_of_week,date,sales,customers,open,promo,state_holiday,school_holiday,store_type,assortment,competition_distance,competition_open_since_month,competition_open_since_year,promo2,promo2_since_week,promo2_since_year,promo_interval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [160]:
df_raw['date']=pd.to_datetime(df_raw['date'])
df_raw['competition_open_since_year']=pd.to_datetime(df_raw['competition_open_since_year'])
df_raw['promo2_since_year']=pd.to_datetime(df_raw['promo2_since_year'])

In [161]:
df_raw.head()

,store,day_of_week,date,sales,customers,open,promo,state_holiday,school_holiday,store_type,assortment,competition_distance,competition_open_since_month,competition_open_since_year,promo2,promo2_since_week,promo2_since_year,promo_interval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,1970-01-01 00:00:00.000002008,0,NaN,NaT,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,1970-01-01 00:00:00.000002007,1,13.0,1970-01-01 00:00:00.000002010,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,1970-01-01 00:00:00.000002006,1,14.0,1970-01-01 00:00:00.000002011,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,1970-01-01 00:00:00.000002009,0,NaN,NaT,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,1970-01-01 00:00:00.000002015,0,NaN,NaT,NaN


In [162]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 18 columns):
 #   Column                        Non-Null Count    Dtype         
---  ------                        --------------    -----         
 0   store                         1017209 non-null  int64         
 1   day_of_week                   1017209 non-null  int64         
 2   date                          1017209 non-null  datetime64[ns]
 3   sales                         1017209 non-null  int64         
 4   customers                     1017209 non-null  int64         
 5   open                          1017209 non-null  int64         
 6   promo                         1017209 non-null  int64         
 7   state_holiday                 1017209 non-null  object        
 8   school_holiday                1017209 non-null  int64         
 9   store_type                    1017209 non-null  object        
 10  assortment                    1017209 non-null  object        
 11

### Checking for Null values

In [163]:
df_raw.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

### Missing values treatment

***competition_distance***

There are 2642 numll values in this column. Since values of competitors distance from shop is not available, we will assume that there is no compititor for the shop. We will fill the null values with a distance value greater than the maximum value in the column.

In [164]:
df_raw.competition_distance.max()

75860.0

I will assume that the compititor for the shops with null values in this column is at the distance of 100000 meters

In [165]:
import numpy as np

In [166]:
df_raw['competition_distance'][119210]==np.nan

False

In [170]:
df_raw['competition_distance'] = df_raw['competition_distance'].fillna(100000.0)

In [171]:
df_raw['competition_distance'].max()

100000.0

In [172]:
df_raw['competition_distance'].isna().sum()

0

# Data Cleaning

# Step 3:  Hypothesis driven EDA

# Step 4: Outlier and missing value Treatment

# Step 5: Feature Engineering

# Step 6: ML solution Frameworks

# Step 7: Choice of Model

# Step 8: Choice of model Validation Framework

# Step 9: Choice of hyperparamer tuning methods

# Step 10: Choice of deployment solution framework

# Step 11: Impact on business problem